In [1]:
%display latex
import sage.misc.banner; sage.misc.banner.banner()

┌────────────────────────────────────────────────────────────────────┐
│ SageMath version 10.7, Release Date: 2025-08-09                    │
│ Using Python 3.12.11. Type "help()" for help.                      │
└────────────────────────────────────────────────────────────────────┘


## Example 3

Here, we compute the isogeny type of ${\bf U}_A$ where $A$ is a certain abelian variety with $a$-number $1$ and prescribed Newton polygon $\lambda$. To be precise, we take $A$ such that
$$M_A=\mathbb{E}/f,\quad f=a_0 F^g+a_1 F^{g-1}+\dots+a_g+a_{g+1}V+\dots+a_{2g}V^g$$
where 
$$a_i=\begin{cases}
p^j, &(i,j) \text{ is a breaking point of }\lambda\text{ and }i\le g\\
p^{j+g-i}, &(i,j) \text{ is a breaking point of }\lambda\text{ and }i> g\\
0, &\text{ otherwise}
\end{cases}$$ 

We test all Polygons $\lambda$ wich contain no slopes equal to $0$, because then the isogeny type of ${\bf U}_A$ is realised by a smaller-dimensional abelian variety. We go up to dimension $g=6$.

In [2]:
p = 3
q = p
K = GF(q)

In [3]:
# convert a square sparse column array into a matrix
def CSCtoMat(ring,csc):
    m = matrix(ring,len(csc),len(csc))
    for i in range(len(csc)):
        for j,v in csc[i]:
            m[j,i]=v
    return m

# multiply a matrix (left) with a sparse column array (right)
def mul(mat,csc):
    Nr,Nc = mat.nrows(),len(csc)
    retval = matrix(mat.base_ring(),Nr,Nc)
    for i in range(Nr):
        for j in range(Nc):
            for k,v in csc[j]:
                retval[i,j] += v*(mat[i,k])
    return retval

# direct sum of square sparse column arrays
def dirSumCSC(csc1,csc2):
    csc = csc1[:]
    for col in csc2:
        csc.append([(el[0]+len(csc1),el[1]) for el in col])
    return csc

def wittFrob(w,W):
    t = list(w.coordinates())
    for i in range(len(t)):
        t[i]=t[i]^p
    return W(t)

def dimension(g,n,F,DFt,A,Apt):
    S = (mul(Apt,DFt)).transpose()-mul(A,F)
    gens = [S[i][j].coordinates()[k] for i in range(2*g) for j in range(2*g) for k in range(n)]
    return ideal(gens).dimension()

We define functions which compute matrices for $F,F^{\vee}$ in the basis $F^{g-1},\dots,1,V,\dots,V^g$, from the datum of the Newton polygon.

In [4]:
def FNewt(g, Newt, WRp, WR1, WR0):
    a = [WR0]*(2*g-1)+[WR1]
    for i in range(1,len(Newt)-1):
        if Newt[i][0]<g+1:
            a[Newt[i][0]-1]=WRp**(Newt[i][1])
        else:
            a[Newt[i][0]-1]=WRp**(Newt[i][1]+g-Newt[i][0])
    #
    F = [[]]+[[((i-1)%(2*g),WR1)] for i in range(1,g)]+[[((i-1)%(2*g),WRp)] for i in range(g,2*g)]
    F[0] += [(i,-a[i]) for i in range(2*g)]
    #
    DFt = [[((i+1)%(2*g),WRp)] for i in range(g-1)]+[[((i+1)%(2*g),WR1)] for i in range(g-1,2*g-1)]+[[]]
    DFt[2*g-1] += [(0,-WRp)]+[(i+1,-WRp*a[i]) for i in range (g-1)]+[(i+1,-a[i])for i in range (g-1,2*g-2)]
    #
    return F,DFt

def dimspecial(g,n,Newt):
    R = PolynomialRing(K,'x',n*g*(2*g-1))
    #finv = K.frobenius_endomorphism(-1)
    # source code in: miniconda3\envs\sage\lib\python3.12\site-packages\sage\rings\padics\witt_vector_ring.py
    WR = WittVectorRing(R, p=p, prec=n, algorithm='finotti')
    WR0,WR1,WRp = WR.zero(),WR.one(),p*WR.one()
    A = matrix(WR,2*g,2*g)
    idx = 0
    for i in range(2*g):
        for j in range(i):
            A[i,j] = WR(R.gens()[n*idx:n*(idx+1)])
            A[j,i] = -A[i,j]
            idx += 1
    del idx
    Apt = A.apply_map(lambda x:wittFrob(x,WR)).transpose()
    F,DFt=FNewt(g,Newt,WRp,WR1,WR0)
    #display(CSCtoMat(WR,F))
    #display(CSCtoMat(WR,DFt))
    return dimension(g,n,F,DFt,A,Apt)

Auxiliary functions for outputting the isogeny type of $\bf{U}_A$

In [5]:
def slopiNewt(Newt):
    slopi = []
    slopim = []
    for i in range(len(Newt)-1):
        l = Rational((Newt[i+1][1]-Newt[i][1])/(Newt[i+1][0]-Newt[i][0]))
        m = gcd(Newt[i+1][1]-Newt[i][1],Newt[i+1][0]-Newt[i][0])
        slopi += [l]*m
        slopim += [[l,m]]
    return slopi, slopim

def dimNewt(Newt):
    slopi = slopiNewt(Newt)[0]
    dpar = g*(g-1)//2
    for i in range(len(slopi)):
        if 2*slopi[i]<1:
            dpar = dpar - (slopi[i].denominator()-2*slopi[i].numerator())*(slopi[i].denominator()-1)//2
        for j in range(i):
            if slopi[i]+slopi[j]<1:
                a1 = slopi[i].numerator()
                b1 = slopi[i].denominator()
                a2 = slopi[j].numerator()
                b2 = slopi[j].denominator()
                dpar = dpar - (b1*b2-a1*b2-a2*b1)
    return dpar

def isogenyType(g,Newt):
    l=[0]
    dmax = dimNewt(Newt)
    for i in range(1,g):
        d = dimspecial(g,i,Newt)
        if d == dmax:
            l = l+[d]
            break    
        else:
            l = l+[d]
    l+=[l[-1]]
    k=[]
    for i in range(0,len(l)-2):
        k += [i+1]*(2*l[i+1]-l[i]-l[i+2])
    return k

The Newton polygons are loaded from an external file. We run through the Newton polygons for $g$ ranging from $3$ to $6$

In [6]:
import json
with open("newton_curves_2_to_20.json","r") as ftemp:
    curves = json.load(ftemp)
g=3
print('g =',3)
listacurve = curves[str(g)]
tab3 = [['Newton polygon','Dimension of UA', 'Isogeny type']]
tab3 += [[slopiNewt(i)[0], dimNewt(i), isogenyType(g,i)] for i in listacurve]
display(table(tab3))

g=4
print('g =',4)
listacurve = curves[str(g)]
tab4 = [['Newton polygon','Dimension of UA', 'Isogeny type']]
tab4 += [[slopiNewt(i)[0], dimNewt(i), isogenyType(g,i)] for i in listacurve]
display(table(tab4))

g=5
print('g =',5)
listacurve = curves[str(g)]
tab5 = [['Newton polygon','Dimension of UA', 'Isogeny type']]
tab5 += [[slopiNewt(i)[0], dimNewt(i), isogenyType(g,i)] for i in listacurve]
display(table(tab5))

g=6
print('g =',6)
listacurve = curves[str(g)]
tab6 = [['Newton polygon','Dimension of UA', 'Isogeny type']]
tab6 += [[slopiNewt(listacurve[0])[0],dimNewt(listacurve[0]),[1,2,3,4,5]]]
tab6 += [[slopiNewt(listacurve[i])[0],dimNewt(listacurve[i]), isogenyType(g,listacurve[i])] for i in range(1,len(listacurve)-1)]
tab6 += [[slopiNewt(listacurve[6])[0],dimNewt(listacurve[6]),[1,1,2,3,4]]]
display(table(tab6))



g = 3


Newton polygon,Dimension of UA,Isogeny type
"\(\left[\frac{1}{3}, \frac{2}{3}\right]\)",\(2\),"\(\left[1, 1\right]\)"
"\(\left[\frac{1}{2}, \frac{1}{2}, \frac{1}{2}\right]\)",\(3\),"\(\left[1, 2\right]\)"


g = 4


Newton polygon,Dimension of UA,Isogeny type
"\(\left[\frac{1}{2}, \frac{1}{2}, \frac{1}{2}, \frac{1}{2}\right]\)",\(6\),"\(\left[1, 2, 3\right]\)"
"\(\left[\frac{1}{4}, \frac{3}{4}\right]\)",\(3\),"\(\left[1, 1, 1\right]\)"
"\(\left[\frac{1}{3}, \frac{1}{2}, \frac{2}{3}\right]\)",\(4\),"\(\left[1, 1, 2\right]\)"


g = 5


Newton polygon,Dimension of UA,Isogeny type
"\(\left[\frac{2}{5}, \frac{3}{5}\right]\)",\(8\),"\(\left[1, 2, 2, 3\right]\)"
"\(\left[\frac{1}{5}, \frac{4}{5}\right]\)",\(4\),"\(\left[1, 1, 1, 1\right]\)"
"\(\left[\frac{1}{2}, \frac{1}{2}, \frac{1}{2}, \frac{1}{2}, \frac{1}{2}\right]\)",\(10\),"\(\left[1, 2, 3, 4\right]\)"
"\(\left[\frac{1}{4}, \frac{1}{2}, \frac{3}{4}\right]\)",\(5\),"\(\left[1, 1, 1, 2\right]\)"
"\(\left[\frac{1}{3}, \frac{1}{2}, \frac{1}{2}, \frac{2}{3}\right]\)",\(7\),"\(\left[1, 1, 2, 3\right]\)"


g = 6


Newton polygon,Dimension of UA,Isogeny type
"\(\left[\frac{1}{2}, \frac{1}{2}, \frac{1}{2}, \frac{1}{2}, \frac{1}{2}, \frac{1}{2}\right]\)",\(15\),"\(\left[1, 2, 3, 4, 5\right]\)"
"\(\left[\frac{1}{3}, \frac{1}{3}, \frac{2}{3}, \frac{2}{3}\right]\)",\(10\),"\(\left[1, 2, 2, 2, 3\right]\)"
"\(\left[\frac{1}{6}, \frac{5}{6}\right]\)",\(5\),"\(\left[1, 1, 1, 1, 1\right]\)"
"\(\left[\frac{2}{5}, \frac{1}{2}, \frac{3}{5}\right]\)",\(12\),"\(\left[1, 2, 2, 3, 4\right]\)"
"\(\left[\frac{1}{5}, \frac{1}{2}, \frac{4}{5}\right]\)",\(6\),"\(\left[1, 1, 1, 1, 2\right]\)"
"\(\left[\frac{1}{4}, \frac{1}{2}, \frac{1}{2}, \frac{3}{4}\right]\)",\(8\),"\(\left[1, 1, 1, 2, 3\right]\)"
"\(\left[\frac{1}{3}, \frac{1}{2}, \frac{1}{2}, \frac{1}{2}, \frac{2}{3}\right]\)",\(11\),"\(\left[1, 1, 2, 3, 4\right]\)"
